# JijModeling with OpenJij

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/OpenJij/OpenJijTutorial/blob/master/source/en/003-PyQUBO_2_OpenJij.ipynb)

In this note, we first explain how to write a mathematical model with JijModeling. After that, we explain how to convert the model to QUBO using JijModeling Transpiler and run simulated annealing with OpenJij.
We solve the clique cover problem as an example.

Please check the documents for a more detailed explanation of JijModeling and JijModeling Transpiler.

We install `jijmodeling` and `jijmodeling-transpiler` by using the pip python package.
Note that `jijmodeling-transpiler` is available with Python 3.9

In [ ]:
!pip install jijmodeling
!pip install jijmodeling-transpiler

In [ ]:
!python --version

In [ ]:
import jijmodeling as jm
import numpy as np
import matplotlib.pyplot as plt

## Formulation of QUBO with JijModeling

JijModeling is an intuitive modeling library for formulating mathematical models, and we can easily convert mathematical models to QUBO. In the last notes, we have shown the case without JijModeling, so we had to formulate QUBO, expand the expressions ourselves, and put them into the Python script. However, we can eliminate those hassles with JijModeling.

JijModeling is a handy library that can help us reduce the computational and implementation errors in our QUBO and Ising model transformations.

Let us solve the clique cover problem as an example.

For more details on this problem, see also [here (T-Wave: creek coverage problem (only Japanese))](https://qard.is.tohoku.ac.jp/T-Wave/?p=434).

We introduce the formulation of the clique cover problem as a mathematical model.

This problem is the computational problem of determining whether, given a graph $G=(V, E)$ and $n$ different colors, the graph can be partitioned into $n$ cliques.

The mathematical model of this problem is:

$$H_B =  \sum^n_{i=1}\left[
\frac{1}{2}\left(-1+\sum_{v \in V} x_{v,i}\right)\sum_{v \in V} x_{v, i} 
- \sum_{(u, v)\in E} x_{u,i} x_{v, i}\right]$$
$$\mathrm{s.t.\  }\quad  \sum^n_{i=1} x_{v, i} = 1\quad \mathrm{forall}\ v$$


Here, $x_{v,i}$ is a binary variable and if the vertex $v$ is colored by the $i$th color, then $x_{v,i} = 1$.
This is the objective function that shows how close the split subgraph is to a clique (complete graph).

The QUBO of this model is as follows:

$$
\begin{aligned}
H &= A H_A + B H_B \\
 &= A\sum_v \left(1-\sum^n_{i=1} x_{v, i}\right)^2 
+ B \sum^n_{i=1}\left[
\frac{1}{2}\left(-1+\sum_{v \in V} x_{v,i}\right)\sum_{v \in V} x_{v, i} 
- \sum_{(u, v)\in E} x_{u,i} x_{v, i}\right]
\end{aligned}$$

The constraint is that only one color is painted on each vertex.
We treat the first term as a penalty term, and the second as a cost (objective function). We want both objective and constraint terms to satisfy being zero.
Normally, we write down the QUBO model to use the Ising optimization. However, JijModeling reduces this process.

We set the graph $G=(V, E)$ and the number of cliques $n$ as follows:

In [2]:
# set the number of vertexes
N_VER = 8
# set the number of colors
N_COLOR = 3
# set the graph. define them which vertices are connected to each other
edges = [[0,1], [0,2], [1,2], [5,6], [2,3], [2,5], [3,4], [5,7], [7, 6]]

### Formulation with JijModeling

In [ ]:
import jijmodeling as jm

First, we prepare the variables for the mathematical model. We set an array of variables using `Array`.
Since we need the `(N_VER) x (N_COLOR)` numbers of variables, we set the `shape` argument as follows.

In [ ]:
problem = jm.Problem('clique')

N = jm.Placeholder('N')
V = jm.Placeholder('V')
E = jm.Placeholder('E',dim = 2)
x = jm.Binary('x', shape=(V,N))
i = jm.Element('i',(0,N))
v = jm.Element('v',(0,V))
e = jm.Element('e',E)

objective = jm.Sum(i, 1/2 * ( -1 + jm.Sum(v,x[v,i]) ) * jm.Sum(v,x[v,i]) - jm.Sum(e,x[e[0],i] * x[e[1],i]))
problem += objective
constraint = jm.Constraint("onehot",jm.Sum(i,x[v,i]) == 1,forall= v)
problem += constraint
problem

As shown, the mathematical model part can be written simply.

Next, we need to prepare the instance data for the placeholder value.

In [ ]:
instance_data = {'N':N_COLOR,'V':N_VER,'E':edges }

We now have prepared the mathematical model and the instance data.

### Converting mathematical model to QUBO

We import the transpiler function to create QUBO from the mathematical model.

In [ ]:
from jijmodeling.transpiler.pyqubo.to_pyqubo import to_pyqubo

In [ ]:
from jijmodeling.transpiler.pyqubo import to_pyqubo

In [ ]:
import jijmodeling.transpiler

In [ ]:
pyq_obj, pyq_cache = to_pyqubo(problem, instance_data,fixed_variables={})

`to_pyqubo()` creates pyqubo object (`pyq_obj`) and the information on the relationship between pyqubo object and the mathematical model is in `pyq_cache`.
Please see the documentation of JijModelingTranspiler for more detailed information about `to_pyqubo()`.

We can easily convert to QUBO (Python dictionary type) with `pyq_obj.compile().to_qubo()`.

In OpenJij and D-Wave Ocean, QUBO is assumed to be represented by a Python dictionary type.

We can run it on each solver by `.compile`.

In [ ]:
# compile this model
qubo, offset = pyq_obj.compile().to_qubo(feed_dict = {'onehot':1.0})

`qubo` is set to QUBO and `offset` is set to the constant that appears when it is converted to QUBO.

## Run with OpenJij

Let us use OpenJij to solve the clique cover problem.

In this tutorial, we use the simulated annealing (SA) solver, but we can also use simulated quantum annealing (SQA).

In [ ]:
# use SA on neal
import openjij as oj
sampler = oj.SASampler()
response = sampler.sample_qubo(qubo)

`.first.sample` extracts the lowest energy of all derived solutions.

In [ ]:
print(response.first.sample)

To make the result more readable, use the `.decode` method.

In [ ]:
result = pyq_cache.decode(response)
result.record.solution

The solution is in the COO format, which means that the solution array contains only indices with values of 1.
This format is useful because we usually only know which value is 1 and which is not.
Let us see the result.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
# initialize vertex color list
node_colors = [-1] * N_VER
# set color list for visualization
colorlist = ['gold', 'violet', 'limegreen', 'darkorange']
# set vertex color list
for node_num,class_num in zip(*result.record.solution['x'][0][0]):
    node_colors[node_num] = colorlist[class_num]
# make figure
fig = plt.figure()
G = nx.Graph()
G.add_nodes_from(range(N_VER))
G.add_edges_from(instance_data["E"])
nx.draw_networkx(G, node_color=node_colors, with_labels=True)

The graph is divided by each clique.

We can check the energy and objective value by using `result.evaluation`

In [ ]:
result.evaluation

## Summary

We learned how to formulate using JijModeling and how it works with OpenJij.

The steps are as follows:

1. Use JijModeling to export the mathematical model.
2. Convert the mathematical model to QUBO using JijModeling Transpiler.
3. Compile QUBO and convert it to a dictionary type.
4. Use OpenJij solvers (such as `SASampler()`) to solve the optimization problem.
5. Decode the solution using `decode`.

JijModeling is a convenient and powerful tool to formulate, and the JijModeling Transpiler provides a simple interface to convert mathematical models to QUBO.
It can also be used with OpenJij, for which a variety of solvers are available, to provide a comfortable development environment.